# IBM Data Science specialization on Coursera

### Flooooor, the Netherlands

This notebook will be used for the final Capstone project of the 9-course specialization in Data Science from IBM on Coursera.  
The goal of this notebook is to determine in which neighborhood in Utrecht a new vegetarian restaurant should be opened. 

# Project description

## 1. Introduction
The beautiful city of Utrecht is based in the Netherlands in Europe.   
It is currently filling up with expats due to multiple factors such as the steep housing prices in Amsterdam, the close proximity to Amsterdam (25 mins by train) and Brexit. As a result, spend on eating/drinking out is rising and so is awareness of animal welfare and evironmental impact of food. This offers opportunities for opening a vegetarian restaurant in Utrecht. In order to determine the best neighborhood to settle, the proximity of restaurants overall and presence of vegetarian restaurants is researched for the neighborhoods of Utrecht. It is assumed that close proximity of other restaurants is favorable, as it will decrease the need for marketing to attract people to a quiet location. 

## 2. Research question
Which neighborhood is most promising for opening a vegetarian restaurant?

In order to solve this question multiple underlying questions will be researched:

* What is the proximity of restaurants in the neighborhood?
* How many vegetarian restaurants have already settled in the neighborhood?

## 3. Data collection
The data used for this project will come from different sources:

* The neighborhoods in Utrecht. [(Utrecht)](https://www.utrecht.nl/wonen-en-leven/wijken/indeling-wijken-en-buurten/buurt-bij-wijk/)
* The postal codes belonging to the neighborhoods. [(Postcode)](https://postcodebijadres.nl/gemeente/utrecht)
* The venues in each neighborhood. [(FourSquare API)](https://developer.foursquare.com/)
* Coordinates for the postal codes. [(Geocoder)](https://geocoder.readthedocs.io/index.html)

The data will be merged so that each postal code is a row containing the neighborhod, district (there are multiple neighborhoods per district), coordinates, inhabitants, number of restaurants and average rating. 

## 4. Notes
This research is limited in the sense that it leaves out factors as investment costs and saturation.  
It focuses on settling where the density of restaurants is high and the presence of vegetarian restaurants is low. 

# I Data preparation

In [2]:
#Import the packages 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysisb
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Install Folium requirements
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

#Install beautifulsoup requirements
!conda install beautifulsoup4
!conda install lxml
!conda install requests

from bs4 import BeautifulSoup
import lxml
import requests

#Install requirements for extracting zipped csv from a url
import zipfile
import urllib.request   
import os

#Install Geocoder requirements
!pip install geocoder
import geocoder

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Scrap data from Utrecht to get the different neighborhoods and districts

In [12]:
url = 'https://www.utrecht.nl/wonen-en-leven/wijken/indeling-wijken-en-buurten/buurt-bij-wijk/'
data = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

In [13]:
neighborhoodlist = []

for row in soup.find_all("tr"):
        cells = row.find_all("td")
        if len(cells) > 0:
            neighborhood = cells[0].text.strip()
            district = cells[1].text.strip()
            
            neighborhoodlist.append((
                neighborhood,
                district
            ))

In [14]:
utrecht_neighborhoods = pd.DataFrame(neighborhoodlist, columns = ['Neighborhood', 'District'])
utrecht_neighborhoods.head()

,Neighborhood,District
0,1e Daalsebuurt,Noordwest
1,2e Daalsebuurt,Noordwest
2,Abstede,Oost
3,Achter 't Spoor,Vleuten-De Meern
4,Adenauerlaan e.o.,Zuidwest


### Scrap data from CBS to get the postal codes belonging to the neighbourhoods

In [15]:
#Load the data from the CBS, which provides separate csv's for municipality, neighborhood and postal codes
url_postcode = 'https://www.cbs.nl/-/media/_excel/2017/38/2017-cbs-pc6huisnr20170801_buurt.zip'
urllib.request.urlretrieve(url_postcode, "2017-cbs-pc6huisnr20170801_buurt.zip")

compressed_file = zipfile.ZipFile('2017-cbs-pc6huisnr20170801_buurt.zip')
postalcode = compressed_file.open('pc6hnr20170801_gwb.csv')
neighborhood = compressed_file.open('buurtnaam2017.csv')
municipality= compressed_file.open('gemeentenaam2017.csv')

#Read the csv's from the zipped file
postalcode = pd.read_csv(postalcode, sep = ';')
neighborhood = pd.read_csv(neighborhood, encoding = "ISO-8859-1", sep=';')
municipality = pd.read_csv(municipality, encoding = "ISO-8859-1", sep=';')

In [16]:
#Correct column names for merge
postalcode.rename(columns={'Gem2017':'GEM2017', 'Buurt2017':'BUURT2017'}, inplace=True)

#Merge neighborhood and municipality name to postalcode
df_postalcodes = pd.merge(postalcode, municipality, how='left')
df_postalcodes = pd.merge(df_postalcodes, neighborhood, how='left')
df_postalcodes.head()

,PC6,Huisnummer,BUURT2017,Wijk2017,GEM2017,GEMNAAM,BUURTNAAM
0,1011AB,105,3630400,36304,363,Amsterdam,Oosterdokseiland
1,1011AB,106,3630400,36304,363,Amsterdam,Oosterdokseiland
2,1011AB,107,3630400,36304,363,Amsterdam,Oosterdokseiland
3,1011AB,110,3630400,36304,363,Amsterdam,Oosterdokseiland
4,1011AB,112,3630400,36304,363,Amsterdam,Oosterdokseiland


In [17]:
#Filter on Utrecht
df_postalcodes2 = df_postalcodes.loc[df_postalcodes['GEMNAAM'] == 'Utrecht']

#Merge with utrecht_neighborhoods
df_postalcodes2.rename(columns={'BUURTNAAM':'Neighborhood'}, inplace=True)
utrecht = pd.merge(df_postalcodes2, utrecht_neighborhoods, how='left')

In [9]:
#Drop columns that are not needed
utrecht = utrecht.drop(['PC6', 'Huisnummer', 'BUURT2017', 'Wijk2017', 'GEM2017'], axis=1)
#Rename Dutch columns
utrecht.rename(columns={'GEMNAAM':'Municipality'}, inplace=True)
#Remove duplicates
utrecht = utrecht.drop_duplicates().reset_index(drop=True)

### Here comes a bit of failed attempts at getting coordinates (this code ended up not being used).
I made several attempts at adding coordinates, see below.  
Unfortunately it turns out that Geocoder doesn't work for the Netherlands and I couldn't get another service to work without paying.  
Therefore I decided to slim down my research to focus on the districts of Utrecht. I'll add the coordinates manually so that I can still use the Foursquare API. 

In [12]:
#First I attempted using Geocoder, but it doesn't work for the Netherlands.

df = utrecht

#Create function to retrieve coordinates for a postal code in Utrecht
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Utrecht'.format(postal_code_from_df))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        return latitude,longitude

#Use the function to fill the latitude and longitude to the df
df['Latitude'], df['Longitude'] = zip(*df['PostalCode'].apply(get_geocoder))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#Another Dutch service is called PDOK, but it has been taken out of service. 
    
def get_coordinates(postalcode, verbose=False):
    try:
        url = 'http://geodata.nationaalgeoregister.nl/geocoder/Geocoder?zoekterm={}'.format(postalcode)
        response = requests.get(url).json()
        if verbose:
            print('Geocode API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] 
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
postalcode = '3451AA'
bloemstraat_utrecht = get_coordinates(postalcode)
print('Coordinate of {}: {}'.format(bloemstraat_utrecht, utrecht))   

In [ ]:
#Another service for the Netherlands called geocode.xyz seemed good, but the json output format doesn't work. 

def get_coordinates(postalcode, verbose=False):
    try:
        url = 'https://geocode.xyz/{}?region=NL?json=1'.format(postalcode)
        response = requests.get(url).json()
        if verbose:
            print('Geocode API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] 
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
postalcode = '3451AA'
bloemstraat_utrecht = get_coordinates(postalcode)
print('Coordinate of {}: {}'.format(bloemstraat_utrecht, utrecht))

In [ ]:
#Final attempt, but here the service timed out as I made too many calls for the free account. Ready to give up now. 

import re
from urllib.request import urlopen
url = 'https://geocode.xyz/3451AA?region=NL?geoit'

conn = urlopen(url)
html = conn.read()

soup = BeautifulSoup(html)
links = soup.find_all('a')

for row in links:
    link = tag.get('href',None).lstrip('https://geocode.xyz')
    link = re.sub('[^0-9,]', "", link)
    link = link.split('\n', 1)[0]
    if link is not None:
        print(link)


### Get coordinates using Nomatim, also not working

In [11]:
geolocator = Nominatim(timeout=3, scheme='http', user_agent="foursquare")

latitude = []
longitude = []

for row in utrecht_neighborhoods:
    neighborhood = utrecht_neighborhoods['Neighborhood']
    if geolocator.geocode(neighborhood) is not None:
        location = geolocator.geocode(neighborhood)
        latitude = location.latitude
        longitude = location.longitude
        latitude.append(latitude)
        longitude.append(longitude)      
    else:
        latitude = None
        latitude = None
        latitude.append(latitude)
        longitude.append(longitude)

utrecht_neighborhoods['Latitude']=latitude
utrecht_neighborhoods['Longitude']=longitude 

GeocoderQueryError: HTTP Error 414: Request-URI Too Long

### Slimming down the dataframe and adding coordinates manually.

In [19]:
#Convert the series to dataframe and keep only unique districts.
utrecht = pd.DataFrame(utrecht)
utrecht_slim = utrecht[['District']].drop_duplicates().dropna().reset_index(drop=True)

In [20]:
#Manually add the coordinates that I got from Google Maps
coordinates = pd.DataFrame(
    [[52.0842733,4.9498756],
    [52.0878183,5.020179],
    [52.0856257,5.1030925],
     [52.0704242,5.0663319],
     [52.1050091,5.1161888],
     [52.0667667,5.1142045],
     [52.0778309,5.1257182],
     [52.1058367,5.0344612],
     [52.1105542,5.070605],
     [52.1202142,5.0923172]],
    columns = ['Latitude', 'Longitude'])

utrecht = pd.merge(utrecht_slim, coordinates, left_index=True, right_index=True)
display(utrecht)

,District,Latitude,Longitude
0,Vleuten-De Meern,52.084273,4.949876
1,Leidsche Rijn,52.087818,5.020179
2,Binnenstad,52.085626,5.103092
3,Zuidwest,52.070424,5.066332
4,Noordoost,52.105009,5.116189
5,Zuid,52.066767,5.114204
6,Oost,52.077831,5.125718
7,West,52.105837,5.034461
8,Noordwest,52.110554,5.070605
9,Overvecht,52.120214,5.092317


### Getting Foursquare data via API

In [21]:
#@hidden_cell
CLIENT_ID = 'REDACTED'
CLIENT_SECRET = 'REDACTED'
VERSION = '20191210'

In [22]:
search_query= 'restaurant'
intent = 'browse'
radius = 1000
latitude=52.085626
longitude=5.103092
limit=50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, limit)

results = requests.get(url).json()

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4cb30d5d562d224b6ade2a88,Croeselaan 303,NL,Utrecht,Nederland,NaN,866,"[Croeselaan 303, 3521 BT Utrecht, Nederland]","[{'label': 'display', 'lat': 52.08040373949391...",52.080404,5.112478,NaN,3521 BT,Utrecht,Restaurant de Branding,v-1575993361,NaN
1,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4bf18d66189f0f472e65b762,Drieharingenstraat 4,NL,Utrecht,Nederland,NaN,1124,"[Drieharingenstraat 4, 3511 BJ Utrecht, Nederl...","[{'label': 'display', 'lat': 52.09199932381554...",52.091999,5.115853,NaN,3511 BJ,Utrecht,Restaurant Vis&Meer,v-1575993361,61786239
2,[],False,4befcb2e24f19c749522f983,Jaarbeursplein 13,NL,Utrecht,Nederland,NaN,381,"[Jaarbeursplein 13, 3521 AS Utrecht, Nederland]","[{'label': 'display', 'lat': 52.088592, 'lng':...",52.088592,5.105886,NaN,3521 AS,Utrecht,Café Restaurant Catharijnehof,v-1575993361,NaN
3,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4e08acd9aeb74c3991177673,Admiraal Helfrichlaan 56,NL,Utrecht,Nederland,NaN,900,"[Admiraal Helfrichlaan 56, 3527 Utrecht, Neder...","[{'label': 'display', 'lat': 52.079791, 'lng':...",52.079791,5.093967,NaN,3527,Utrecht,Restaurant Oriënt,v-1575993361,NaN
4,"[{'id': '4bf58dd8d48988d1c3941735', 'name': 'M...",False,4c288d74ce3fc92830446d88,Kanaalstraat 83,NL,Utrecht,Nederland,NaN,650,"[Kanaalstraat 83, 3531 CC Utrecht, Nederland]","[{'label': 'display', 'lat': 52.091423, 'lng':...",52.091423,5.101842,NaN,3531 CC,Utrecht,Petit Restaurant Royal,v-1575993361,NaN


In [25]:
df = pd.DataFrame()

for ind in utrecht.index:
    neighborhood = utrecht['District'][ind]
    lat = utrecht['Latitude'][ind]
    lon = utrecht['Longitude'][ind]
    categoryId = '4d4b7105d754a06374d81259' # Category food
    limit=100
    radius = 1000
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&categoryId={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lon, categoryId, VERSION, radius, limit)
    results = requests.get(url).json()
    new_venues = results['response']['venues']
    if len(new_venues) > 0:
        new_venues = json_normalize(new_venues)
        new_venues['District'] = neighborhood
        df = pd.concat([df, new_venues], ignore_index=True, sort=False)
    else:
        print('no venues for district {}'.format(neighborhood))

df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,District,location.neighborhood,venuePage.id
0,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",False,50159481e4b06a029efa9b47,NaN,NL,Harmelen,Nederland,NaN,616,"[3481 Harmelen, Nederland]","[{'label': 'display', 'lat': 52.08498592402544...",52.084986,4.958819,3481,Utrecht,Stadhoudershof,v-1575993375,Vleuten-De Meern,NaN,NaN
1,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",False,4bbb6006b35776b0e4dfcb01,NaN,NL,NaN,Nederland,NaN,917,[Nederland],"[{'label': 'display', 'lat': 52.080557, 'lng':...",52.080557,4.937897,NaN,NaN,Restaurant Samsam,v-1575993375,Vleuten-De Meern,NaN,NaN
2,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",False,4e5bd5d87d8b966a15e80598,Jaagpad,NL,Harmelen,Nederland,NaN,994,"[Jaagpad, Harmelen-City, Nederland]","[{'label': 'display', 'lat': 52.090669, 'lng':...",52.090669,4.960031,NaN,Utrecht,Appelboom,v-1575993375,Vleuten-De Meern,NaN,NaN
3,"[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",False,4b59c115f964a5204b9628e3,Kloosterweg 2,NL,Harmelen,Nederland,NaN,1209,"[Kloosterweg 2, 3481 XC Harmelen, Nederland]","[{'label': 'display', 'lat': 52.09069635700111...",52.090696,4.964145,3481 XC,Utrecht,De Kloosterhoeve,v-1575993375,Vleuten-De Meern,NaN,NaN
4,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4d93a4079892a1cd48c98f69,NaN,NL,Harmelen,Nederland,NaN,1197,"[Harmelen, Nederland]","[{'label': 'display', 'lat': 52.09100280094515...",52.091003,4.963541,NaN,Utrecht,Cafetaria Vikas,v-1575993375,Vleuten-De Meern,NaN,NaN


In [26]:
#Check total restaurants per neighborhood to see if the limit for Foursquare needs to be >100. Not necessary
grouped_df = df.groupby(['District']).count()[['categories']]
display(grouped_df)

,categories
District,
Binnenstad,50
Leidsche Rijn,41
Noordoost,50
Noordwest,33
Oost,50
Overvecht,29
Vleuten-De Meern,19
West,39
Zuid,50


In [27]:
df_vegetarian = pd.DataFrame()

for ind in utrecht.index:
    neighborhood = utrecht['District'][ind]
    lat = utrecht['Latitude'][ind]
    lon = utrecht['Longitude'][ind]
    categoryId = '4bf58dd8d48988d1d3941735' # Category vegetarian
    limit =100
    radius = 1000
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&categoryId={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lon, categoryId, VERSION, radius, limit)
    results = requests.get(url).json()
    new_venues = results['response']['venues']
    if len(new_venues) > 0:
        new_venues = json_normalize(new_venues)
        new_venues['District'] = neighborhood
        df_vegetarian = pd.concat([df_vegetarian, new_venues], ignore_index=True, sort=False)
    else:
        print('no vegetarian venues for district {}'.format(neighborhood))

df_vegetarian.head()

no vegetarian venues for district Vleuten-De Meern
no vegetarian venues for district Leidsche Rijn
no vegetarian venues for district Zuidwest
no vegetarian venues for district Zuid
no vegetarian venues for district Noordwest
no vegetarian venues for district Overvecht


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id,District,location.crossStreet
0,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,5c4a3d91916bc1002ca7004d,Vredenburg 28,NL,Utrecht,Nederland,1083,"[Vredenburg 28, 3511 BC Utrecht, Nederland]","[{'label': 'display', 'lat': 52.093145, 'lng':...",52.093145,5.113162,NaN,3511 BC,Utrecht,Waku Waku,v-1575993379,560619300,Binnenstad,NaN
1,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,5a77479218d43b7c90f40ede,NaN,NL,Utrecht,Nederland,1107,"[3511 BN Utrecht, Nederland]","[{'label': 'display', 'lat': 52.092635, 'lng':...",52.092635,5.114588,NaN,3511 BN,Utrecht,Ekki,v-1575993379,NaN,Binnenstad,NaN
2,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,5b0eca02c21cb1002cc6106e,Vredenburg 28,NL,Utrecht,Nederland,1053,"[Vredenburg 28, 3511BC Utrecht, Nederland]","[{'label': 'display', 'lat': 52.092945, 'lng':...",52.092945,5.112867,Wijk C,3511BC,Utrecht,Snckbr IV,v-1575993379,NaN,Binnenstad,NaN
3,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,5436b677498e8dcd303d8b12,Hoog Catharijne,NL,Utrecht,Nederland,1043,"[Hoog Catharijne, Utrecht, Nederland]","[{'label': 'display', 'lat': 52.091405, 'lng':...",52.091405,5.115107,NaN,NaN,Utrecht,Rammenas,v-1575993379,NaN,Binnenstad,NaN
4,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,4c4f0ea692b6a5932854f470,Antonius Matthaeuslaan 112,NL,Utrecht,Nederland,315,[Antonius Matthaeuslaan 112 (Jacob van der Bor...,"[{'label': 'display', 'lat': 52.10476046786198...",52.104760,5.120784,NaN,3515 AV,Utrecht,Soy,v-1575993380,NaN,Noordoost,Jacob van der Borchstraat


In [28]:
grouped_df_vegetarian = df_vegetarian.groupby(['District']).count()[['categories']]
display(grouped_df_vegetarian)

,categories
District,
Binnenstad,4
Noordoost,2
Oost,1
West,1


In [29]:
#Only keep id and vegetarian flag for vegetarian restaurants. 
temp = df_vegetarian[['id']]
temp['Vegetarian'] = 1
temp.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,id,Vegetarian
0,5c4a3d91916bc1002ca7004d,1
1,5a77479218d43b7c90f40ede,1
2,5b0eca02c21cb1002cc6106e,1
3,5436b677498e8dcd303d8b12,1
4,4c4f0ea692b6a5932854f470,1


In [31]:
#Combine the two dataframes to identify vegetarian restaurants
df = pd.merge(df, temp, how='left')
df['is_vegetarian'] = df['Vegetarian']>0
# df = df.astype({"Vegetarian": int})
df.dtypes

categories                    object
hasPerk                         bool
id                            object
location.address              object
location.cc                   object
location.city                 object
location.country              object
location.crossStreet          object
location.distance              int64
location.formattedAddress     object
location.labeledLatLngs       object
location.lat                 float64
location.lng                 float64
location.postalCode           object
location.state                object
name                          object
referralId                    object
District                      object
location.neighborhood         object
venuePage.id                  object
Vegetarian                   float64
is_vegetarian                   bool
dtype: object

# Create map that shows the neighborhoods and restaurants. Red restaurants are vegetarian. 

In [81]:
utrecht_center = [52.0894444, 5.1077981]

map_utrecht = folium.Map(location=utrecht_center, zoom_start=13)
for lat, lng, neighborhood in zip(utrecht['Latitude'], utrecht['Longitude'], utrecht['District']):
    folium.Marker([lat,lng], popup='{}'.format(neighborhood)).add_to(map_utrecht)
    folium.Circle(
        [lat, lng], 
        radius=1000, #A radius of the 1km that was used to check existing restaurants via Foursquare
        color='black', 
        fill=False, 
        fill_color='white', 
        fill_opacity=3).add_to(map_utrecht)
for lat, lng, neighborhood, vegetarian in zip(df['location.lat'], df['location.lng'], df['District'], df['is_vegetarian']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    color='red' if vegetarian else 'grey'
    folium.CircleMarker(
        [lat, lng], 
        radius=3, 
        color=color, 
        fill=True, 
        fill_color=color, 
        fill_opacity=1).add_to(map_utrecht)
    
map_utrecht

In [78]:
summary = df.groupby(df['District']).agg({'id': 'count', 'Vegetarian': 'sum'}).reset_index().rename(columns={'id':'Total_restaurants', 'Vegetarian':'Vegetarian_restaurants'})
summary = summary.sort_values(['Total_restaurants', 'Vegetarian_restaurants'], ascending=False)
summary.head()

,District,Total_restaurants,Vegetarian_restaurants
2,Noordoost,50,2.0
0,Binnenstad,50,1.0
4,Oost,50,1.0
8,Zuid,50,0.0
1,Leidsche Rijn,41,0.0


### Deep dive
It would be useful to get ratings for the existing 8 vegetarian restaurants in order to better choose a suitable neighborhood to settle in. 
Unfortunately, it seems a paid account is necessary in order to get these details from Foursquare. 

In [61]:
vegetarian_ratings = pd.DataFrame()

for id in zip(df_vegetarian['id']):
    VENUE_ID = id
    url = 'https://api.foursquare.com/v2/venues/?VENUE_ID={}&client_id={}&client_secret={}&v={}'.format(VENUE_ID, CLIENT_ID, CLIENT_SECRET, VERSION)
    results = requests.get(url).json()
    rating = results['response']
    if len(rating) > 0:
        rating = json_normalize(rating)
        rating['Rating'] = rating
        vegetarian_ratings = pd.concat([vegetarian_ratings, rating], ignore_index=True, sort=False)
    else:
        print('no rating available for venue {}'.format(VENUE_ID))

vegetarian_ratings.head()

no rating available for venue ('5c4a3d91916bc1002ca7004d',)
no rating available for venue ('5a77479218d43b7c90f40ede',)
no rating available for venue ('5b0eca02c21cb1002cc6106e',)
no rating available for venue ('5436b677498e8dcd303d8b12',)
no rating available for venue ('4c4f0ea692b6a5932854f470',)
no rating available for venue ('5583e720498e0b1f54ed9887',)
no rating available for venue ('53de0a36498ed10c9ea8b2dd',)
no rating available for venue ('4bcb58320687ef3b0e86ddcc',)


""


# Results and Discussion

The analysis gives insight in the current situation of restaurant density per neighborhood in Utrecht. It shows the few existing vegetarian restaurants plotted on the map, so infrastructure and overlap of neigborhoods can be taken into account. 

Unfortunately it cost a lot of effort to retrieve the coordinates and a manual solution was necessary. This shows the lack of availability of reverse geocoding tools in the Netherlands and took away a lot of time for this project. In addition, the original approach to include data on restaurant ratings was not feasible as Foursquare does not openly share this via their API. Therefore the eventual project ended up more limited than originally planned. 

# Conlusion

Based on the research, Utrecht Oost (East) is an attractive location to start a new vegetarian restaurant. Only one vegetarian restaurant is already settled there and as it is not the city center (binnenstad) finding a good location will be much easier. Additionally it is close to the central station and other central attractions. 